<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/observability/grafana_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set variables

In [ ]:
project = "demo_project"
worker_name = "worker-02"

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

openrc_path = "/content/" + project + "-openrc.sh"

with open(openrc_path, 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools, get_available_publicips, get_segment_ids, get_radio_interfaces, get_worker_interfaces

# Reserve resources

In the next cells, we reserve 1 worker.

In [ ]:
# reserve worker
worker_lease = show_reservation_byname(worker_name+"-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

# Define network

In the following section we setup the edge-net network.

In [ ]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip=None,enable_dhcp=False)
logger.success("edge-net is created.")

# Run containers

Run a container for Grafana, with a public IP

In [ ]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
if len(available_ifs) < 2:
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find enough interfaces on {worker_name}")

# run container
publicnet = chi.network.get_network("serverpublic")
edgenet = chi.network.get_network("edge-net")
container_name = "grafana"
chi.container.create_container(
    name=container_name,
    image="grafana/grafana",
    reservation_id=worker_reservation_id,
    environment={
        "GF_INSTALL_PLUGINS": "grafana-mqtt-datasource",  # Install MQTT plugin
        "GF_SECURITY_ADMIN_USER": "default",  # Set admin username
        "GF_SECURITY_ADMIN_PASSWORD": "defaultdefault",  # Set admin password
    },
    mounts=[],
    nets=[
        {"network": publicnet['id']},
        {"network": edgenet['id']},
    ],
    labels={
        "networks.1.interface": available_ifs[0],
        "networks.1.ip": pub_ip + "/27",
        "networks.2.interface": available_ifs[1],
        "networks.2.ip": "10.70.70.210/24",
        "capabilities.privileged": "true",
    },
    command=["sh","-c","ip route del default && ip route add default via 130.237.11.97 && exec /run.sh"]
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}")